# Автокодировщики. Раунд второй. 

На паре мы немного потренировались в создании автокодировщиков. Мы выяснили, что в случае цифр не все пиксели оказываются полезными. Давайте теперь сделаем всё то же самое, но с человеческими лицами из [lfw-датасета.](http://vis-www.cs.umass.edu/lfw/)

Качать датасет отсюда: 

* http://www.cs.columbia.edu/CAVE/databases/pubfig/download/lfw_attributes.txt
* http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz
* http://vis-www.cs.umass.edu/lfw/lfw.tgz

In [ ]:
import tensorflow as tf
import keras, keras.layers as L, keras.backend as K
import numpy as np
from sklearn.model_selection import train_test_split

# подгружаем функцию для предобработки датасета
from lfw_dataset import load_lfw_dataset
%matplotlib inline
import matplotlib.pyplot as plt
import download_utils
import keras_utils
import numpy as np
from keras_utils import reset_tf_session

In [4]:
from sklearn.model_selection import train_test_split
X, attr = load_lfw_dataset(use_raw=True,dimx=38,dimy=38)
X = X.astype('float32') / 256.0

img_shape = X.shape[1:]

X_train, X_test = train_test_split(X, test_size=0.1,random_state=42)

FileNotFoundError: File b'lfw_attributes.txt' does not exist

In [ ]:
X_train[0].shape

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.title('sample image')
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(X[i])

print("X shape:",X.shape)
print("attr shape:",attr.shape)

In [ ]:
import tensorflow as tf
import keras, keras.layers as L
s = keras.backend.get_session()

Делай раз! Давайте соберём линейный автокодировщие, иными словами говоря PCA и посмотрим что из этого выйдет. 

In [ ]:
def build_pca_autoencoder(img_shape,code_size=32):
    """
    Here we define a simple linear autoencoder as described above.
    We also flatten and un-flatten data to be compatible with image shapes
    """
    
    encoder = keras.models.Sequential()
    encoder.add(L.InputLayer(img_shape))
    encoder.add(L.Flatten())                  #flatten image to vector
    encoder.add(L.Dense(code_size))           #actual encoder

    decoder = keras.models.Sequential()
    decoder.add(L.InputLayer((code_size,)))
    decoder.add(L.Dense(np.prod(img_shape)))  #actual decoder, height*width*3 units
    decoder.add(L.Reshape(img_shape))         #un-flatten
    
    return encoder,decoder

Объединим их в одну модель.

In [ ]:
encoder,decoder = build_pca_autoencoder(img_shape,code_size=32)

inp = L.Input(img_shape)
code = encoder(inp)
reconstruction = decoder(code)

autoencoder = keras.models.Model(inp,reconstruction)
autoencoder.compile('adamax','mse')

In [ ]:
autoencoder.fit(x=X_train,y=X_train,epochs=10,
                validation_data=[X_test,X_test])

In [ ]:
def visualize(img,encoder,decoder):
    """Draws original, encoded and decoded images"""
    code = encoder.predict(img[None])[0]
    reco = decoder.predict(code[None])[0]

    plt.subplot(1,3,1)
    plt.title("Original")
    plt.imshow(img)

    plt.subplot(1,3,2)
    plt.title("Code")
    plt.imshow(code.reshape([code.shape[-1]//2,-1]))

    plt.subplot(1,3,3)
    plt.title("Reconstructed")
    plt.imshow(reco.clip(0,1))
    plt.show()


In [ ]:
score = autoencoder.evaluate(X_test,X_test,verbose=0)
print("Final MSE:",score)

for i in range(5):
    img = X_test[i]
    visualize(img,encoder,decoder)
